<a href="https://colab.research.google.com/github/Afsah-Hyder/Computer-Vision-Course/blob/main/CV_Assignment_02_ah07065.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Computer Vision Assigment 02
## Name: Afsah Hyder
## ID: ah07065

**Question 02** In this question, you will have to read up a bit on the different types of GANS that are found:
Style transfer GAN, CGANS, CycleGAN, SRGANS. Only reading the abstract and introduction
of these papers will equip you to solve these questions, reading them fully is up to you. Then, go through the given situations and suggest the suitable GAN along with a short but
suitable explanation:



---


a. A researcher gave a student the job to perform some preprocessing techniques on an image dataset. The student, while playing around with that data, applies a sufficiently large median blurring kernel to the images, but deletes the original files. These new images were padded correctly as to retain the original size of the images. Which GAN is best suited to revive the original HD images and why? [10]

Super-Resolution GANs seem to be most suitable for this.Super-resolution GAN applies a deep network in combination with an adversary network to produce higher resolution images. As shown above, SRGAN is more appealing to a human with more details compared with the similar design without GAN (SRResNet).



---


b. Interns at a computer vision company are tasked with adding different skin textures to
images of pandas and regular bears. If presented with a picture of a regular bear, the
skin should be changed to match that of a panda. They do not have paired images of
pandas and bears that they can use as direct mappings of each other. They have a set
of images of pandas, and a different set of images of bears. Which GAN is best suited to
achieve this image-to-image translation task and why? [10]

CycleGAN



---


c. A daughter wants to give her Van Gogh-fanatic mother a present. So, she decides to
turna set of family photos to a set that looks as if it's been painted by Van Gogh. Which
GAN's best suited to achieve this task and how would she train her model (using what
datasets, etc.)? [10]

Style Transfer GAN



